# Subsea Inpanting using Hifill ([paper](https://arxiv.org/abs/2005.09704), [code](https://github.com/Atlas200dk/sample-imageinpainting-HiFill))
 - It will be necessary to allow Internet Access at the Kaggle notebook settings
 - It is recommended to activate GPU accelerator at the Kaggle notebook settings 

# 1. Setup

## 1.1. Install older tensorflow

In [ ]:
!pip install tensorflow==1.15.2 

## 1.2. Install [viajen](https://github.com/brunomsantiago/viajen) for animation visualization

In [ ]:
!pip install viajen

## 1.3. Download Hifill code and models from [subsea inpainting repository](https://github.com/brunomsantiago/subsea_inpainting)

In [ ]:
!git clone https://github.com/brunomsantiago/subsea_inpainting --branch models

## 1.4. Normal imports

In [ ]:
# Standard Library imports
from pathlib import Path
import time
import sys

# Third party imports
import cv2
from IPython.core.display import HTML
import numpy as np
from PIL import Image
import tensorflow as tf
from viajen import animate

## 1.5 Local import (non-installed module)

In [ ]:
sys.path.insert(1, './subsea_inpainting/code/hifill')
from hifill import inpaint

# 2. Frames to be inpainted (+ masks)

## 2.1. Select clip to work with

In [ ]:
data_name = '01a'
# data_name = '01b'
# data_name = '01c'
# data_name = '02a'
# data_name = '03a'
# data_name = '03b'
# data_name = '04a'
# data_name = '05a'
# data_name = '05b'
# data_name = '05c'
# data_name = '05d'
# data_name = '05e'
# data_name = '05f'

## 2.2. Mount folders

In [ ]:
base_input_folder = '../input/subsea-inpainting-dataset/'
base_output_folder = './'

frames_folder = Path(base_input_folder) / 'prepared_images' / data_name / 'frames'
masks_folder = Path(base_input_folder) / 'prepared_images' / data_name / 'masks'

results_folder = Path(base_output_folder) / 'results'/ 'hifill' / data_name

## 2.3. View clip

In [ ]:
animate.folder(frames_folder, max_height=200)

In [ ]:
animate.folder(masks_folder, max_height=200)

# 3. Inpaiting

## 3.1 Weights locations

In [ ]:
hifill_weights  = './subsea_inpainting/models_weights/hifill/hifill.pb'

## 3.2. Function to apply inpainting

In [ ]:
def inpaint_hifill(frames_folder, masks_folder, out_folder, model_path):
    start_time = time.time()
    print('Starting...')
    frames_files = sorted(Path(frames_folder).glob('*.png'))
    masks_files = sorted(Path(masks_folder).glob('*.png'))
    out_folder.mkdir(parents=True, exist_ok=True)
    
    with tf.Graph().as_default():
        with open(model_path, 'rb') as f:
            output_graph_def = tf.GraphDef()
            graph_string = f.read()
            output_graph_def.ParseFromString(graph_string)
            tf.import_graph_def(output_graph_def, name="")

        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)
            image_ph = sess.graph.get_tensor_by_name('img:0')
            mask_ph = sess.graph.get_tensor_by_name('mask:0')
            inpainted_512_node = sess.graph.get_tensor_by_name('inpainted:0')
            attention_node = sess.graph.get_tensor_by_name('attention:0')
            mask_512_node = sess.graph.get_tensor_by_name('mask_processed:0')

            for frame_file, mask_file in zip(frames_files, masks_files):
                print(frame_file.name)
                frame = cv2.imread(frame_file.as_posix())
                mask = cv2.imread(mask_file.as_posix())
                mask = 255 - mask
                inpainted_frame = inpaint(frame,
                                        mask,
                                        sess,
                                        inpainted_512_node,
                                        attention_node,
                                        mask_512_node,
                                        image_ph,
                                        mask_ph,
                                        6)
                filepath = (out_folder / frame_file.name).as_posix()
                cv2.imwrite(filepath, inpainted_frame)
    return time.time() - start_time

## 3.3 Execute inpainting

In [ ]:
execution_time = inpaint_hifill(frames_folder,
                                masks_folder,
                                results_folder,
                                hifill_weights)

print(f'\n\n--- Execution time: {execution_time:.2f} seconds ---')

# 4. Show results

In [ ]:
display(HTML('<h2>Original Frames<h2>'))
display(animate.folder(frames_folder))

display(HTML(f'<br><br><h2>Inpainted Frames ({execution_time:.1f} seconds processing)<h2>'))
display(animate.folder(results_folder))